# <center> ISBN Web scraping </center>

- rédiger code pour automatiser la récupération des infos
- quelles infos
- stocker dans un txt
- transformer txt en csv
= bdd

In [1]:
import pandas as pd
import numpy as np
import requests

C:\Users\utilisateur\anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [2]:
# URL
url_bu = 'https://bu-catalogue.univ-brest.fr/cgi-bin/koha/opac-detail.pl?biblionumber=300000'

# URI-SUFFIX
uri_pages = '?page='

# Number of pages to read
nbPages = 10

# XPath content to collect
tags = ['//dt class="labelxslt"', '//h2', '//dt class="labelxslt"', '//dt class="labelxslt"', '//dt class="labelxslt"']
cols = ['Auteur(s)', 'Titre', 'Edition', 'Sujet(s)', 'EAN' ]

#page = requests.get(url)
#doc = lh.fromstring(page.content)

def getPage(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)

    # Get the Web data via XPath
    content = []
    for i in range(len(tags)):
        content.append(doc.xpath(tags[i]))

    # Gather the data into a Pandas DataFrame array
    df_liste = []
    for j in range(len(tags)):
        tmp = pd.DataFrame([content[j][i].text_content().strip() for i in range(len(content[i]))], columns=[cols[j]])
        tmp['key'] = tmp.index
        df_liste.append(tmp)

    # Build the unique Dataframe with one tag (xpath) content per column
    liste = df_liste[0]
    for j in range(len(tags)-1):
        liste = liste.join(df_liste[j+1], on='key', how='left', lsuffix='_l', rsuffix='_r')
        liste['key'] = liste.index
        del liste['key_l']
        del liste['key_r']
    
    return liste

def getPages(_nbPages,_url):
    liste_finale = pd.DataFrame()
    for i in range (_nbPages):
        liste = getPage(_url + uri_pages + str(i+1))
        liste_finale = pd.concat([liste_finale, liste], ignore_index=True)
    return liste_finale

In [2]:
# df = getPages(nbPages,url_bu)

## Web scraping

In [2]:
import requests
url = 'https://bu-catalogue.univ-brest.fr/cgi-bin/koha/opac-detail.pl?biblionumber=361715'
page = requests.get(url)
# Voir le code html source
# print(page.content)

In [3]:
import requests
from bs4 import BeautifulSoup
url = 'https://bu-catalogue.univ-brest.fr/cgi-bin/koha/opac-detail.pl?biblionumber=361715'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
soup.title.string

'Catalogue Ma bibliothèque › Détails de Pierre Vidal-Naquet'

In [13]:
# Trouver tous les éléments avec la balise <a>
soup.find_all('a')

[<a class="brand" href="/cgi-bin/koha/opac-main.pl">
   Ma bibliothèque </a>,
 <a href="#" id="cartmenulink" role="button" title="Sélectionnez les exemplaires qui vous intéressent">
 <i class="fa fa-shopping-cart fa-icon-white" id="carticon"></i> <span class="cartlabel">Panier</span> <span id="basketcount"></span>
 </a>,
 <a class="dropdown-toggle" data-toggle="dropdown" href="#" id="listsmenu" role="button" title="Afficher les listes"><i class="fa fa-list fa-icon-white"></i> <span class="listslabel">Listes</span> <b class="caret"></b></a>,
 <a href="/cgi-bin/koha/opac-shelves.pl?op=list&amp;category=2" role="menuitem" tabindex="-1"><strong>Listes publiques</strong></a>,
 <a href="/cgi-bin/koha/opac-shelves.pl?op=view&amp;shelfnumber=830&amp;sortfield=title" role="menuitem" tabindex="-1">Un été tout en légèreté</a>,
 <a href="/cgi-bin/koha/opac-shelves.pl?op=view&amp;shelfnumber=355&amp;sortfield=title" role="menuitem" tabindex="-1">Sélection du moment</a>,
 <a href="/cgi-bin/koha/opac

In [14]:
#Trouver tous les éléments p avec la classe « title »
soup.find_all("p", class_="title")

[]

In [17]:
soup.find_all("dt", class_="labelxslt")

[<dt class="labelxslt">Auteur(s)</dt>,
 <dt class="labelxslt">ISBN</dt>,
 <dt class="labelxslt">EAN</dt>,
 <dt class="labelxslt">Langue(s)</dt>,
 <dt class="labelxslt">Publication</dt>,
 <dt class="labelxslt">Description</dt>,
 <dt class="labelxslt">Note(s) et résumé(s)</dt>,
 <dt class="labelxslt">Sujet(s)</dt>,
 <dt class="labelxslt">Autres titres en lien</dt>,
 <dt class="labelxslt" style="display:none;">098e</dt>]

#### Titre

In [20]:
soup.find_all('h2')

[<h2>Pierre Vidal-Naquet : une vie / François Dosse </h2>]

In [27]:
soup.find_all("dt", class_="labelxslt")

[<dt class="labelxslt">Auteur(s)</dt>,
 <dt class="labelxslt">ISBN</dt>,
 <dt class="labelxslt">EAN</dt>,
 <dt class="labelxslt">Langue(s)</dt>,
 <dt class="labelxslt">Publication</dt>,
 <dt class="labelxslt">Description</dt>,
 <dt class="labelxslt">Note(s) et résumé(s)</dt>,
 <dt class="labelxslt">Sujet(s)</dt>,
 <dt class="labelxslt">Autres titres en lien</dt>,
 <dt class="labelxslt" style="display:none;">098e</dt>]

In [29]:
soup.find_all("div", id="opacxslt")

[<div id="opacxslt">
 <h2>Pierre Vidal-Naquet : une vie / François Dosse </h2>
 <dl>
 <dt class="labelxslt">Auteur(s)</dt>
 <dd>
 <a href="/cgi-bin/koha/opac-search.pl?q=an:70958">Dosse, François</a>
 <span class="fctxslt"> (Auteur)</span>
 <span class="fctxslt"> (1950-....)</span>
 </dd>
 <dt class="labelxslt">ISBN</dt>
 <dd>978-2-7071-9421-3 </dd>
 <dt class="labelxslt">EAN</dt>
 <dd>9782707194213 </dd>
 <dt class="labelxslt">Langue(s)</dt>
 <dd>français</dd>
 <dt class="labelxslt">Publication</dt>
 <dd>Paris : La Découverte, DL 2020</dd>
 <dt class="labelxslt">Description</dt>
 <dd>1 vol. (660 p.-[16] p. de pl.) : ill., portr., fac-sim., couv. ill. en coul. ; 24 cm. </dd>
 <dt class="labelxslt">Note(s) et résumé(s)</dt>
 <dd>
 <a class="voir_resume" href="#descriptions">Plus d'infos</a>
 </dd>
 <dt class="labelxslt">Sujet(s)</dt>
 <span class="subjectxslt">
 <dd><a href="
      /cgi-bin/koha/opac-search.pl?idx=su,phr&amp;q=Vidal-Naquet+Pierre">Vidal-Naquet</a>, Pierre (1930-2006) | 

## Titre

In [5]:
soup.h2.string

'Pierre Vidal-Naquet : une vie / François Dosse '

In [6]:
titre = soup.h2.string
titre = titre.split("/")[0]
print(titre)

Pierre Vidal-Naquet : une vie 


## Récupérer EAN

In [7]:
# liste pour accueillir chaque EAN 
liste_EAN = []

# trouve l'EAN
EAN =soup.find_all("dd")
print("EAN brut :", EAN[2])

# récupère uniquement les chiffres de l'EAN
# ajout à la liste
for x in EAN[2] :
    liste_EAN.append(int(x))

# visualisation de la liste
print(liste_EAN)

EAN brut : <dd>9782707194213 </dd>
[9782707194213]


## Récupérer le ou les auteurs

Problème = prénom avant le nom, compliqué pour recherche par ordre alphabétique

In [8]:
auteur = soup.h2.string
auteur = auteur.split("/ ")[1]
print(auteur)

François Dosse 


## Récupérer l'édition

In [100]:
edition = soup.find_all("dd")
for x in edition[4]:
    edition_complete = x
print(edition_complete)

Paris : La Découverte, DL 2020


In [132]:
import re
maison_edition = re.split(': |,',edition_complete)
maison_edition = maison_edition[1]
print(maison_edition)

La Découverte


In [101]:
ville = edition_complete.split(" ")[0]
ville

'Paris'

In [110]:
date = edition_complete.split(" ")[-1]
date

'2020'

## Récupérer la langue

In [12]:
langue = soup.h2.string
print(langue)
langue = langue.split("/ ")[1]
print(langue)

Pierre Vidal-Naquet : une vie / François Dosse 
François Dosse 


In [27]:
# trouve l'EAN
langue =soup.find_all("dd")
print("langue :", langue[3])

# # récupère uniquement les chiffres de l'EAN
# # ajout à la liste
# for x in langue[3] :
# #     liste_EAN.append(int(x))
#     print(x)

# # visualisation de la liste


langue : <dd>français</dd>


In [28]:
langue =soup.find_all("dd")
fr = langue[3]

print(fr)

<dd>français</dd>


In [26]:
langue[3]

<dd>français</dd>

## Code

In [272]:
# Importations 
import requests
from bs4 import BeautifulSoup

# page 
url = 'https://bu-catalogue.univ-brest.fr/cgi-bin/koha/opac-detail.pl?biblionumber=308272'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')



# Variables globales
liste_EAN = []
liste_titres = []
auteurs = []
edition = []
date = []
ville = []


# récupérer le titre
titre = soup.h2.string
titre = titre.split("/")[0]
# liste_titres.append(titre)


# récupérer l'EAN 
EAN =soup.find_all("dd")

# récupère uniquement les chiffres de l'EAN
# ajout à la liste
for x in EAN[2] :
#     liste_EAN.append(int(x))
    EAN = x
    
# Récupérer auteur(s)
auteur = soup.h2.string
auteur = auteur.split("/")[1]
    
    
# Récupérer l'édition
edition = soup.find_all("dd")
for x in edition[4]:
    edition_complete = x

import re
maison_edition = re.split(': |,',edition_complete)
maison_edition = maison_edition[1]


# Récupérer la ville
ville = edition_complete.split(" ")[0]

# Récupérer la date
date = edition_complete.split(" ")[-1]

In [275]:
print(auteur+", "+titre+", "+maison_edition+", "+ville+", "+date+", "+EAN)


 Johann Chapoutot , La loi du sang : penser et agir en nazi , Gallimard, Paris, 2014, 9782070141937 


## Récupérer infos et les ajouter à un fichier txt

In [276]:
biblio = str(auteur+", "+titre+", "+maison_edition+", "+ville+", "+date+", "+EAN+"\n")

In [277]:
with open('biblio.txt', 'a') as f:
    f.write(biblio)